In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as pl
import corner
import pymc as pm
import arviz as az
import pandas as pd
import seaborn

In [ ]:
%matplotlib widget

In [ ]:
df = pd.ExcelFile("./Data/Speed Skating Gravity V2.xlsx").parse('10,000m M')

In [ ]:
df2 =pd.DataFrame({"altitude":[], "gravity":[], "year":[], "finish_time":[], "type":[]})

In [ ]:
for index, row in df.iterrows():
    if index == 0:
        continue
    # get colums we are interested in
    alt = row['Unnamed: 4']
    year = row['Unnamed: 3']
    grav = row['Unnamed: 5']

    # if value is a nan then ignore it
    if np.isnan(alt) or np.isnan(year) or np.isnan(grav):
        continue

    # only select a few of the events
    ftime_names = ["Olympics", 'World Single Distances','World Allrounds']
    ftime = None
    event = ""
    for ft in ftime_names:
        if np.isnan(row[ft]):
            continue
        else:
            ftime = row[ft]
            event = ft
            break
    if ftime is None:
        continue
    else:
        df2 = df2.append({"altitude":alt, "gravity":grav, "year":year, "finish_time":ftime, "type":event},ignore_index=True)

In [ ]:
#df2 = pd.read_csv("./spk_10000_dat.csv")

In [ ]:
# ignore the one event with gravity around 8.8 (not sure what this one is)
dff = df2[df2["gravity"] > 9.0]

In [ ]:
dff

In [ ]:
# have a look at all the data to see if any obvious correlations by eye
seaborn.pairplot(dff,  vars=["altitude", "gravity", "year", "finish_time"])

In [ ]:
fig, ax = plt.subplots()
ax.plot(dff["gravity"], dff["finish_time"],".")
#ax.plot(df2["gravity"], df2["finish_time"],".")

In [ ]:
fig, ax = plt.subplots()
ax.plot(dff["altitude"], dff["finish_time"],".")

In [ ]:
fig, ax = plt.subplots()
ax.plot(dff["year"], dff["finish_time"],".")

In [ ]:
#df2.to_csv("./spk_10000_dat.csv")

In [ ]:
def data_model(xg,xa,xy,mg,ma,my,c):
    """Defines the model of a plane with the output being the finish time

    Args:
        xg (_type_): xpositions for gravity
        xa (_type_): xposiitions for altitude
        xy (_type_): x positions for year
        mg (_type_): gradient for gravity
        ma (_type_): gradient for altitude
        my (_type_): gradient for year
        c (_type_): offset

    Returns:
        _type_: finish time
    """
    return mg*xg + ma*xa + my*xy +c

Create the pymc model and run mcmc to get the posterior distribution

In [ ]:
aoff = 700
goff = 9.8
yoff = 2010
with pm.Model() as gauss_mod:
    # shift x values so offset is defines at the center of dataset
    # this can make the posteior easier to sample (more gaussian less correlation)
    xgrav = dff["gravity"] - goff
    xalt = dff["altitude"] - aoff
    xyear = dff["year"] - yoff
    # uniform priors on each of the parameters (very broad priors)
    pr_mg = pm.Uniform("mg",-1000,1000)
    pr_c = pm.Uniform("c",-1000,1000)
    pr_ma = pm.Uniform("ma",-1000,1000)
    pr_my = pm.Uniform("my",-1000,1000)
    # variance of noise
    pr_sigma = pm.Uniform("sigma",0,100)

    model = data_model(xgrav,xalt, xyear,pr_mg, pr_ma, pr_my, pr_c)
    # Gaussian likelihood 
    lik = pm.Normal("lik", mu=model, sigma=pr_sigma, observed = np.squeeze(dff["finish_time"]))

    # setup sampler and generate samples from posterior 
    #step = pm.Slice()
    mcmc_samples = pm.sample(2000)#, step=step)

In [ ]:
# create an array of posterior samples 
samps = np.array([
    np.concatenate(np.array(mcmc_samples.posterior.mg)), 
    np.concatenate(np.array(mcmc_samples.posterior.ma)), 
    np.concatenate(np.array(mcmc_samples.posterior.my)),
    np.concatenate(np.array(mcmc_samples.posterior.c))])

In [ ]:
# plot corner plot of each parameter
az.plot_pair(mcmc_samples, marginals=True)

In [ ]:
# plot marginal posteriors foreach parameter
az.plot_posterior(mcmc_samples)

In [ ]:
fig, ax = plt.subplots()
xdats = np.linspace(9.79, 9.82, 100)
ydats = xdats*619 + 791.5 
ax.plot(xdats, ydats)

In [ ]:
ling = np.linspace(min(dff["gravity"] - goff),max(dff["gravity"] - goff), 10)
lina = np.linspace(min(dff["altitude"] - aoff),max(dff["altitude"] - aoff), 10)
liny = np.linspace(min(dff["year"] - yoff),max(dff["year"] - yoff), 10)

x,y,z = np.meshgrid(ling, lina, liny, indexing="ij")
alldat = np.zeros((len(samps.T), len(ling), len(lina), len(liny)))
for i, sample in enumerate(samps.T):
    val = data_model(x,y,z, *sample)
    alldat[i,:] = np.array(val)

In [ ]:
alldat[:,0].flatten()

In [ ]:
quantsg = np.zeros((len(ling), 3))
for i, v in enumerate(ling):
    quantsg[i] = np.percentile(alldat[:,i].flatten(), [5,50,95])

quantsa = np.zeros((len(lina), 3))
for i, v in enumerate(lina):
    quantsa[i] = np.percentile(alldat[:,:,i].flatten(), [5,50,95])

quantsy = np.zeros((len(liny), 3))
for i, v in enumerate(liny):
    quantsy[i] = np.percentile(alldat[:,:,:,i].flatten(), [5,50,95])

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=(10,11))
ax[0].plot(dff["gravity"], dff["finish_time"],".")
ax[0].plot(ling + goff, quantsg[:,1], color="C1")
ax[0].fill_between(ling + goff, quantsg[:,0], quantsg[:,2], color="C1", alpha=0.5)
ax[0].set_xlabel("gravity (g) [ms^-2]")
ax[0].set_ylabel("finish time [s]")

ax[1].plot(dff["altitude"], dff["finish_time"],".")
ax[1].plot(lina + aoff, quantsa[:,1], color="C1")
ax[1].fill_between(lina + aoff, quantsa[:,0], quantsa[:,2], color="C1", alpha=0.5)
ax[1].set_xlabel("altitude [m]")
ax[1].set_ylabel("finish time [s]")

ax[2].plot(dff["year"], dff["finish_time"],".")
ax[2].plot(liny + yoff, quantsy[:,1], color="C1")
ax[2].fill_between(liny+yoff, quantsy[:,0], quantsy[:,2], color="C1", alpha=0.5)
ax[2].set_xlabel("year")
ax[2].set_ylabel("finish time [s]")

In [ ]:
fig, ax = plt.subplots()
ax.plot(dff["altitude"], dff["finish_time"],".")
ax.plot(lina+aoff, quantsa[:,1], color="C1")
ax.fill_between(lina+aoff, quantsa[:,0], quantsa[:,2], color="C1", alpha=0.5)
ax.set_xlabel("altitude [m]")
ax.set_ylabel("finish time [s]")

In [ ]:
fig, ax = plt.subplots()
ax.plot(dff["year"], dff["finish_time"],".")
ax.plot(liny+yoff, quantsy[:,1], color="C1")
ax.fill_between(liny+yoff, quantsy[:,0], quantsy[:,2], color="C1", alpha=0.5)
ax.set_xlabel("year")
ax.set_ylabel("finish time [s]")

In [ ]:
fig, ax = plt.subplots(nrows=1, figsize=(10,12), subplot_kw={'projection': '3d'})
ax.plot(dff["gravity"], dff["altitude"], dff["finish_time"],".")
ax.set_xlabel("gravity (g) [ms^-2]")
ax.set_zlabel("finish time [s]")

